In [1]:
import pandas as pd
import numpy as np
import dill, pickle
import copy
from joblib import Parallel, delayed
import os, sys

from collections import Counter
import itertools
from scipy import stats

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

from skrebate import ReliefF, MultiSURF, MultiSURFstar
from sklearn.feature_selection import f_classif
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn import metrics
from sklearn.metrics import adjusted_rand_score, rand_score
from sklearn.metrics.cluster import pair_confusion_matrix

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import LeaveOneOut

In [2]:
data = pd.read_excel('../data/GC-MS_data.xlsx')

In [3]:
ID_num = np.where(data.ID == 'Healthy', 0, 1)

In [4]:
data.insert(1, 'ID_num', ID_num)

In [5]:
data.head()

,ID,ID_num,M151T1,M83T1,M101T1,M46T1,M49T1,M66T1,M62T1,M80T1,...,M110T26_1,M121T26_1,M175T26_1,M124T26_1,M138T26_2,M85T26_2,M174T26,M123T26_1,M91T26,M94T26
0,Healthy,0,16.048388,6064.292377,781.993514,802.020931,4694.204132,7.292742e+04,29491.232768,16.024791,...,7867.025232,8965.565729,3012.051770,4658.714325,2340.026084,12912.976187,1397.742432,16170.724464,10359.088145,4524.513159
1,Healthy,0,2966.261917,134774.774889,62570.597315,18605.033364,23763.284044,6.268676e+06,392830.377721,5189.617648,...,278936.880991,323372.411031,79971.903683,215630.572047,79703.417540,524443.694141,58370.370030,714891.705277,330847.552709,229032.808630
2,Healthy,0,8377.442132,54888.286191,7574.412552,24452.053496,8865.881271,8.122816e+05,470305.922774,2883.659359,...,330297.442948,400547.227331,89001.852491,237332.546382,92068.988978,281554.565002,73030.972156,942769.300372,653550.454878,189904.532666
3,Healthy,0,37.609095,10440.984838,26.750624,88.923190,1827.264205,4.491165e+04,8166.159143,1374.437507,...,1090.182759,998.434986,220.276019,648.220851,375.745446,1799.311194,106.666198,1713.346412,2279.341711,655.383157
4,Healthy,0,115.438628,7287.725513,112.578791,28.656514,5843.718655,1.573132e+05,21056.733832,3441.212011,...,9453.592618,17333.718556,1887.790859,8186.847903,3752.941527,10702.205663,2548.940100,33661.377548,16464.790716,7288.223731


In [6]:
labels = data.ID_num.values
X_all_df = data.drop(['ID','ID_num'], axis = 1)

In [7]:
X_all_df.shape

(43, 2734)

In [8]:
X_all_df.head()

,M151T1,M83T1,M101T1,M46T1,M49T1,M66T1,M62T1,M80T1,M137T1,M135T1,...,M110T26_1,M121T26_1,M175T26_1,M124T26_1,M138T26_2,M85T26_2,M174T26,M123T26_1,M91T26,M94T26
0,16.048388,6064.292377,781.993514,802.020931,4694.204132,7.292742e+04,29491.232768,16.024791,1710.311517,610.786947,...,7867.025232,8965.565729,3012.051770,4658.714325,2340.026084,12912.976187,1397.742432,16170.724464,10359.088145,4524.513159
1,2966.261917,134774.774889,62570.597315,18605.033364,23763.284044,6.268676e+06,392830.377721,5189.617648,1061.179026,63189.361143,...,278936.880991,323372.411031,79971.903683,215630.572047,79703.417540,524443.694141,58370.370030,714891.705277,330847.552709,229032.808630
2,8377.442132,54888.286191,7574.412552,24452.053496,8865.881271,8.122816e+05,470305.922774,2883.659359,4927.060920,68984.802047,...,330297.442948,400547.227331,89001.852491,237332.546382,92068.988978,281554.565002,73030.972156,942769.300372,653550.454878,189904.532666
3,37.609095,10440.984838,26.750624,88.923190,1827.264205,4.491165e+04,8166.159143,1374.437507,732.478087,1881.366829,...,1090.182759,998.434986,220.276019,648.220851,375.745446,1799.311194,106.666198,1713.346412,2279.341711,655.383157
4,115.438628,7287.725513,112.578791,28.656514,5843.718655,1.573132e+05,21056.733832,3441.212011,170.831961,284.265511,...,9453.592618,17333.718556,1887.790859,8186.847903,3752.941527,10702.205663,2548.940100,33661.377548,16464.790716,7288.223731


In [25]:
#DO ONLY ONCE
# with open('./X_df.pik', "wb") as f:
#     pickle.dump(X_all_df, f,protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
def corr_fs(X_df):
    '''Correlation based feature selection (Eliminates features with |corr| > 0.95)'''
    X_arr = X_df.values
    
    correlations = {}
    col_names = X_df.columns.tolist()

    col_ids = list(range(len(col_names)))
    for col_a, col_b in itertools.combinations(col_ids, 2):
        corr,_ = stats.pearsonr(X_arr[:, col_a], X_arr[:, col_b])

        correlations[col_names[col_a] + '__' + col_names[col_b]] = corr

    corr_df = pd.DataFrame.from_dict(correlations, orient='index').reset_index()
    corr_df.columns = ['feature_pair', 'correlation']
    corr_df['abs_correlation'] = np.abs(corr_df.correlation)

    highly_correlated_pair = corr_df.loc[corr_df.abs_correlation > 0.95]

    feat_pair = highly_correlated_pair.feature_pair.values
    feat_pair_corr = highly_correlated_pair.values
    
    #Drop the second feature in every highly correlated pair
    feat_drop = [i.split('__')[1] for i in feat_pair]
    feat_keep = [i.split('__')[0] for i in feat_pair]

    feat_drop_idx = [list(col_names).index(i) for i in feat_drop]
        
    X_no_corr_arr = np.delete(X_arr, feat_drop_idx, axis=1)
    X_no_corr_cols = np.delete(col_names, feat_drop_idx)
    
    X_no_corr_df = pd.DataFrame(X_no_corr_arr, columns=X_no_corr_cols)


    return highly_correlated_pair, X_no_corr_df

In [10]:
loo = LeaveOneOut()

In [11]:
def pre_process(train_index, test_index):
        train_x_df = X_all_df.iloc[train_index, :]        
        _, train_x_no_corr_df = corr_fs(train_x_df)
        col_keep = train_x_no_corr_df.columns
        
        train_x = train_x_no_corr_df.values
        train_y = labels[train_index]
        
        test_x = X_all_df.loc[test_index][col_keep]
        test_y = labels[test_index]
        
        #Scale the data
        scaler = MinMaxScaler()
        train_x = scaler.fit_transform(train_x)
        test_x = scaler.transform(test_x)
    

        return train_x,train_y,test_x,test_y,col_keep

    
cross_val_data = Parallel()(delayed
                    (pre_process)(train_index, test_index)
                        for train_index, test_index in loo.split(X_all_df.values, labels))

In [12]:
#DO THIS ONLY ONCE
# with open('./ml_cv_data.pik', "wb") as f:
#     pickle.dump(cross_val_data, f,protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
#TO READ IT IN
with open('../src/data/ml_cv_data.pik', "rb") as f:
    cv_data = dill.load(f)